In [ ]:
#| default_exp detector

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations

import torch
from fastcore.basics import patch
from torch.nn.functional import normalize

In [ ]:
#| exporti
try:
    import pytorch3d
except ModuleNotFoundError:
    install = "https://github.com/facebookresearch/pytorch3d/blob/main/INSTALL.md"
    raise ModuleNotFoundError(
        f"PyTorch3D is not installed, which is required to parameterize camera poses. See installation instructions here: {install}."
    )

In [ ]:
#| export
class Detector(torch.nn.Module):
    """Construct a 6 DoF X-ray detector system. This model is based on a C-Arm."""

    def __init__(
        self,
        sdr: float,  # Source-to-detector radius (half of the source-to-detector distance)
        height: int,  # Height of the X-ray detector
        width: int,  # Width of the X-ray detector
        delx: float,  # Pixel spacing in the X-direction
        dely: float,  # Pixel spacing in the Y-direction
        n_subsample: int | None = None,  # Number of target points to randomly sample
        reverse_x_axis: bool = False,  # If pose includes reflection (in E(3) not SE(3)), reverse x-axis
    ):
        super().__init__()
        self.sdr = sdr
        self.height = height
        self.width = width
        self.delx = delx
        self.dely = dely
        self.n_subsample = n_subsample
        if self.n_subsample is not None:
            self.subsamples = []
        self.reverse_x_axis = reverse_x_axis

        # Initialize the source and detector plane in default positions (along the x-axis)
        source, target = self._initialize_carm()
        self.register_buffer("source", source)
        self.register_buffer("target", target)

In [ ]:
#| export
@patch
def _initialize_carm(self: Detector):
    """Initialize the default position for the source and detector plane."""
    # Initialize the source on the x-axis
    source = torch.tensor([[self.sdr, 0.0, 0.0]])

    # Initialize the center of the detector plane on the negative x-axis
    center = torch.tensor([[-self.sdr, 0.0, 0.0]])

    # Use the standard basis for the detector plane
    basis = torch.tensor([[0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])

    # Construct the detector plane with different offsets for even or odd heights
    h_off = 1.0 if self.height % 2 else 0.5
    w_off = 1.0 if self.width % 2 else 0.5

    # Construct equally spaced points along the basis vectors
    t = (torch.arange(-self.height // 2, self.height // 2) + h_off) * self.delx
    s = (torch.arange(-self.width // 2, self.width // 2) + w_off) * self.dely
    if self.reverse_x_axis:
        s = -s
    coefs = torch.cartesian_prod(t, s).reshape(-1, 2)
    target = torch.einsum("cd,nc->nd", basis, coefs)
    target += center

    # Batch source and target
    source = source.unsqueeze(0)
    target = target.unsqueeze(0)

    if self.n_subsample is not None:
        sample = torch.randperm(self.height * self.width)[: int(self.n_subsample)]
        target = target[:, sample, :]
        self.subsamples.append(sample.tolist())
    return source, target

`DiffDRR` can take a rotation parameterized in any of the following forms to move the detector plane:
- `axis_angle`
- `euler_angles` (note: will also need to specify the `convention` for the Euler angles)
- `matrix`
- `quaternion`
- `rotation_6d`

In [ ]:
#| exporti
PARAMETERIZATIONS = [
    "axis_angle",
    "euler_angles",
    "matrix",
    "quaternion",
    "rotation_6d",
    "rotation_10d",
]

In [ ]:
#| exporti
from pytorch3d.transforms import (
    axis_angle_to_matrix,
    euler_angles_to_matrix,
    quaternion_to_matrix,
    rotation_6d_to_matrix,
)


def _convert_to_rotation_matrix(rotations, parameterization, convention):
    """Convert any parameterization of a rotation to a matrix representation."""
    if parameterization == "axis_angle":
        R = axis_angle_to_matrix(rotations)
    elif parameterization == "euler_angles":
        R = euler_angles_to_matrix(rotations, convention)
    elif parameterization == "quaternion":
        R = quaternion_to_matrix(rotations)
    elif parameterization == "rotation_6d":
        R = rotation_6d_to_matrix(rotations)
    else:
        raise ValueError(
            f"parameterization must be in {PARAMETERIZATIONS}, not {parameterization}"
        )
    return R

In [ ]:
#| export
from pytorch3d.transforms import Transform3d


@patch
def forward(
    self: Detector,
    rotations: torch.Tensor,  # Some (batched) representation of a rotation
    translations: torch.Tensor,  # Batch of C-arm translations (bx, by, bz)
    parameterization: str,  # Specifies the representation of the rotation
    convention: str,  # If parameterization is Euler angles, specify convention
):
    """Create source and target points for X-rays to trace through the volume."""
    if parameterization not in PARAMETERIZATIONS:
        raise ValueError(
            f"parameterization must be in f{PARAMETERIZATIONS}, not {parameterization}"
        )
    if parameterization == "rotation_10d":
        raise NotImplementedError(
            "rotation_10d is not supported yet, but will be in the future"
        )
    if parameterization == "euler_angles" and convention is None:
        raise ValueError(
            "convention for Euler angles must be specified as a 3 letter combination of [X, Y, Z]"
        )

    # Convert rotation representation to a rotation matrix, R
    # Transpose R to convert to right-handed convention for PyTorch3D
    R = _convert_to_rotation_matrix(rotations, parameterization, convention)
    R = R.transpose(-1, -2)
    t = Transform3d(device=rotations.device).rotate(R).translate(translations)
    source = t.transform_points(self.source)
    target = t.transform_points(self.target)
    return source, target

In [ ]:
#| export
# TODO: Provide way to convert any rotation to DeepDRR angles

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()